In [1]:
import pandas as pd
import os
from nltk.tokenize import word_tokenize
from pyvi import ViTokenizer, ViPosTagger
from nltk import pos_tag

In [120]:
path = ".\\Bahnar_dataset"
folder = "Gia Lai"

READ DICTIONARY

In [121]:
dict_path = path + "\\Gia Lai Dictionary.xlsx"
dict_frame =pd.read_excel(dict_path)

dict_frame.head(5)

,language0,language1,word_type
0,Ái ân,Tơ hưch dih băl,d
1,Ái tình,Hưch băl,d
2,A lô,Alô,đg
3,Ăm,Amĭn,đg
4,Ăm em,Amĭn oh; pôk,đg


In [122]:
print(dict_frame.size)

45090


In [166]:
def push_word(dict, word, value):
    if word in dict:
        # Check for tag
        not_Contain = 1
        for tag in dict[word][1]:
            if tag == value[1]:
                not_Contain = 0
                break
        if not_Contain == 1:
            dict[word][1].add(value[1])
            dict[word][0].add(value[0])
            # print(f"{word} with Tag: {len(dict[word][1])}")            
    else:
        dict[word] = [set(), set()]
        dict[word][0].add(value[0])
        dict[word][1].add(value[1])

tag_dict = {
    'A':'t',
    'C':'k',
    'E':'E',
    'I':'I',
    'L':'L',
    'M':'M',
    'N':'d',
    'Nc':'Nc',
    'Ny':'Ny',
    'Np':'Np',
    'Nu':'Nu',
    'P':'đ',
    'R':'p',
    'S':'k',
    'T':'tr',
    'V':'đg',
    'X':'x'
}
# Convert pyvi library Tag to Bahnar Tag
def converter(char):

    if char in tag_dict:
        return tag_dict[char]
    else:
        return char

In [168]:
dict = {}
for row in dict_frame.iterrows():

    bana_word = row[1]['language1'].lower()
    bana_word = bana_word.strip()

    vietnam_word = row[1]['language0']

    if(type(vietnam_word) != type(str())):
        continue
    else:
        vietnam_word = vietnam_word.lower().strip()
    vietnam_word = vietnam_word.strip()        


    # In case we just have VietNam token but do not have bahnar token => Use VietNam token to find Bahnar token
    if(row[1].isnull().any()):
        token, tag = ViPosTagger.postagging(ViTokenizer.tokenize(vietnam_word))
        if len(token) == 1:
            # Các từ Bahnar có cùng một tiếng Việt
            for item in bana_word.split(';'):
                push_word(dict, item.strip(), [vietnam_word, converter(tag[0])])

    # In case we have both of them, then we just push them to the dictionary
    else:
        tag = row[1]['word_type'].lower()
        # Các từ Bahnar có cùng một tiếng Việt
        for item in bana_word.split(';'):
            push_word(dict, item.strip(), [vietnam_word, tag])
    # break

print(dict['tơ hưch dih băl'])
print(len(dict))

[{'ái ân'}, {'d'}]
6600


READ DATASET

In [169]:
dataset = []
file_name = []
dataset = []
dataset_path = path + "\\" + folder
for obj in os.scandir(dataset_path):
    dataframe = pd.read_excel(dataset_path + "\\" + obj.name)
    dataset.append((obj.name, dataframe))

In [170]:
a, b = dataset[0]
print(a)
print(b.head(5))

Bok Set phat rung 246-483.xlsx
                                          Tiếng BANA  \
0                                                240   
1     Hơ tôch nỡr Giông pơma, pơk bâr bok PơlăJă ...   
2                           Manat j at kơ ih õng Inh   
3  Bok Pơlă Jă Jal kuăr kơ dih Giông, păng nhâm p...   
4  Sỡ ki nhi nhăp pỗ Set pơm bôl juăt đe hơnăn pỗ...   

                                          TIẾNG VIỆT  
0                                                246  
1      Giông vừa nói xong tới ông Pơlă Jă Jal nói...  
2                           Tội nghiệp con rể của ta  
3  Ông Pơlă Jă Jal ôm lấy Giông bật khóc nức nờ v...  
4  Ngày xưa ta với bạn Set kết nghĩa anh em thân ...  


TOKENIZING

In [204]:
class Sequence_Info():
    def __init__(self, bana_token, bana_tag, vietnam_token, vietnam_tag, miscount) -> None:
        self.bana_token = bana_token
        self.bana_tag = bana_tag

        self.vietnam_token = vietnam_token
        self.vietnam_tag = vietnam_tag

        self.miscount = miscount

    def get_bana(self):
        return self.bana_token
    
    def get_vietnamese(self):
        return self.vietnam_token
    
    def get_miscount(self):
        return self.miscount
    
    def write_excel(self, fp):
        fp.write(f"{self.miscount}\n")

        token_str = self.bana_token[0]
        tag_str = '/'.join(self.bana_tag[0])
        for token, tag in zip(self.bana_token[1:], self.bana_tag[1:]):
            tag_str = tag_str + "," + '/'.join(tag)
            token_str = token_str + "," + token

        fp.write(f"{token_str}\n")
        fp.write(f"{tag_str}\n")

        fp.write(f"{','.join(self.vietnam_token)}\n")
        fp.write(f"{','.join(self.vietnam_tag)}\n")
        

In [210]:
def token_tag_vietnam(sequence):
    token, tag = ViPosTagger.postagging(ViTokenizer.tokenize(sequence))
    return token, tag

def token_tag_bahnar(sequence):
    token_list = []
    tag_list = []

    words = word_tokenize(sequence)
    miscount = 0
        
    i = 0
    while(i < len(words)):
        j = 1
        while (i + j <= len(words)):
            if ' '.join(words[i : i + j]) in dict:
                j += 1
            else:
                break
            
        # In case this word is in dict => write into a file and then mark the pos tag for it
        if j > 1:
            token = ' '.join(words[i : i + j - 1])

            token_list.append(token)
            tag_list.append(dict[token][1])

            i += (j-1)
            
        # In case the token is not in dict => write the token and mark it
        else:
            miscount += 1
            token_list.append(words[i])
            tag_list.append(" ")

            i += 1

    return token_list, tag_list, miscount
    

In [211]:
def sort_by_missword(info):
    return info.miscount

In [214]:
contain_file = f"{folder}_info"
if not os.path.isdir(f"{path}\\{contain_file}"):
    os.makedirs(f"{path}\\{contain_file}")

# LOOPING ALL FILES IN 1 FOLDER
for file_name, contend in dataset:
    sentence_lst = []

    # LOOPING ALL SENTENCE IN 1 FILE EXCEL
    for idx, paragraph in contend.iterrows():

        bana_sentence = paragraph['Tiếng BANA']
        vietnam_sentence = paragraph['TIẾNG VIỆT']

        # Checking for all cell do not get value
        if type(bana_sentence) != type(str()) or type(vietnam_sentence) != type(str()):
            continue

        viet_token, viet_tag = token_tag_vietnam(vietnam_sentence.strip())

        bana_token, bana_tag, miscount = token_tag_bahnar(bana_sentence.strip())

        # Checking for all sentence cannot get token
        if len(viet_token) <= 0 or len(bana_token) <= 0:
            continue

        seq_info = Sequence_Info(bana_token, bana_tag, viet_token, viet_tag, miscount)
        sentence_lst.append(seq_info)

    # Sorting the sequence info following the number of missing words
    sentence_lst.sort(key=sort_by_missword)

    # Write in the csv file
    fstream = open(path + "\\" + contain_file + "\\" + file_name.split('.')[0] + ".csv", 'w+', encoding='utf-8')
    for i in sentence_lst:
        i.write_excel(fstream)
        fstream.write("\n")

    fstream.close()

In [216]:
print(token_tag_vietnam(" Được, không sao đâu cháu".strip()))

(['Được', ',', 'không', 'sao', 'đâu', 'cháu'], ['V', 'F', 'R', 'P', 'R', 'N'])
